In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("Final.db")
cursor = conn.cursor()
cursor.execute("SELECT * FROM raw_stroke_data")
cursor.fetchall()

[('id',
  'gender',
  'age',
  'hypertension',
  'heart_disease',
  'ever_married',
  'work_type',
  'Residence_type',
  'avg_glucose_level',
  'bmi',
  'smoking_status',
  'stroke'),
 ('9046',
  'Male',
  '67',
  '0',
  '1',
  'Yes',
  'Private',
  'Urban',
  '228.69',
  '36.6',
  'formerly smoked',
  '1'),
 ('51676',
  'Female',
  '61',
  '0',
  '0',
  'Yes',
  'Self-employed',
  'Rural',
  '202.21',
  'N/A',
  'never smoked',
  '1'),
 ('31112',
  'Male',
  '80',
  '0',
  '1',
  'Yes',
  'Private',
  'Rural',
  '105.92',
  '32.5',
  'never smoked',
  '1'),
 ('60182',
  'Female',
  '49',
  '0',
  '0',
  'Yes',
  'Private',
  'Urban',
  '171.23',
  '34.4',
  'smokes',
  '1'),
 ('1665',
  'Female',
  '79',
  '1',
  '0',
  'Yes',
  'Self-employed',
  'Rural',
  '174.12',
  '24',
  'never smoked',
  '1'),
 ('56669',
  'Male',
  '81',
  '0',
  '0',
  'Yes',
  'Private',
  'Urban',
  '186.21',
  '29',
  'formerly smoked',
  '1'),
 ('53882',
  'Male',
  '74',
  '1',
  '1',
  'Yes',
  'Privat

In [4]:
df = pd.read_sql_query("SELECT * FROM raw_stroke_data", conn)
df.head()

,field1,field2,field3,field4,field5,field6,field7,field8,field9,field10,field11,field12
0,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
1,9046,Male,67,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,51676,Female,61,0,0,Yes,Self-employed,Rural,202.21,N/A,never smoked,1
3,31112,Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
4,60182,Female,49,0,0,Yes,Private,Urban,171.23,34.4,smokes,1


In [31]:
df.columns = df.iloc[0]
df1 = df.drop(df.index[[0]])
df1.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
1,9046,Male,67,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,51676,Female,61,0,0,Yes,Self-employed,Rural,202.21,N/A,never smoked,1
3,31112,Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
4,60182,Female,49,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
5,1665,Female,79,1,0,Yes,Self-employed,Rural,174.12,24,never smoked,1


In [32]:
df1.shape

(5110, 12)

In [33]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5110 entries, 1 to 5110
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5110 non-null   object
 1   gender             5110 non-null   object
 2   age                5110 non-null   object
 3   hypertension       5110 non-null   object
 4   heart_disease      5110 non-null   object
 5   ever_married       5110 non-null   object
 6   work_type          5110 non-null   object
 7   Residence_type     5110 non-null   object
 8   avg_glucose_level  5110 non-null   object
 9   bmi                5110 non-null   object
 10  smoking_status     5110 non-null   object
 11  stroke             5110 non-null   object
dtypes: object(12)
memory usage: 519.0+ KB


In [42]:
# Convert data type to integers and float
df1['age'] = df1['age'].astype(int)
df1['avg_glucose_level'] = df1['avg_glucose_level'].astype(float)
df1['bmi'] = df1['bmi'].astype(float)

ValueError: could not convert string to float: 'N/A'

In [43]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5110 entries, 1 to 5110
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   object 
 1   gender             5110 non-null   object 
 2   age                5110 non-null   int32  
 3   hypertension       5110 non-null   object 
 4   heart_disease      5110 non-null   object 
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                5110 non-null   object 
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   object 
dtypes: float64(1), int32(1), object(10)
memory usage: 499.0+ KB


In [44]:
df1.isnull().sum()

0
id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64